In [10]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import torch.utils
import torch.utils.data

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.log_softmax(x, dim=1)
    
model = Net()

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

In [6]:
epochs = 15
for epoch in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print(f"Epoch {epoch+1} - Training loss: {running_loss/len(trainloader)}")

Epoch 1 - Training loss: 0.2833613711458915
Epoch 2 - Training loss: 0.21875051049185967
Epoch 3 - Training loss: 0.17373955531566065
Epoch 4 - Training loss: 0.14502466530767458
Epoch 5 - Training loss: 0.12659950844391482
Epoch 6 - Training loss: 0.10983200153824427
Epoch 7 - Training loss: 0.09624115138975924
Epoch 8 - Training loss: 0.08680825722791048
Epoch 9 - Training loss: 0.0790055300037601
Epoch 10 - Training loss: 0.07214857054564522
Epoch 11 - Training loss: 0.06579299052970718
Epoch 12 - Training loss: 0.06027371510873233
Epoch 13 - Training loss: 0.055953812485348295
Epoch 14 - Training loss: 0.051210187623021146
Epoch 15 - Training loss: 0.04765980615035327


In [7]:
correct_count, all_count = 0, 0
for images,labels in testloader:
  for i in range(len(labels)):
    with torch.no_grad():
        logps = model(images[i].view(1, 28*28))

    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9767


In [8]:
input_image = torch.rand(1, 1, 28, 28)
output = model(input_image)
print(output)

tensor([[-3.9246, -1.7116, -3.6224, -1.0102, -3.2342, -3.1128, -3.8040, -1.5291,
         -3.8976, -2.7235]], grad_fn=<LogSoftmaxBackward0>)


In [11]:
torch.save(model.state_dict(), 'mnist_model.pth')